In [3]:
import scrapy
from scrapy.crawler import CrawlerProcess
import numpy as np 
from scipy.sparse import csc_matrix
import sys

# create list contain dicts {url_web_wiki: list_url}
wiki_pages = []
node = 1

class TestSpider(scrapy.Spider):
    name = 'test'
    start_urls = [
         'https://en.wikipedia.org/wiki/Information_system'
    ]
    
    def parse(self, response):
        global wiki_pages
        global node
        yield {
            'url': response.request.url, 
            'title': response.xpath("//title/text()").extract_first()
        }

        list_page = response.xpath("//a[starts-with(@href,'/wiki') and not(contains(@href,':'))]/@href").extract()
        dict = {response.request.url: list_page}
        wiki_pages.append(dict)
        # for link in list_page:
        #     if (link.startswith('/wiki/')):
        #         print("https://en.wikipedia.org" + link)
        #         nodes = nodes + 1
        if (list_page is not None) and node < 500:
            node = node + len(list_page)
            for next_page in list_page:
                yield response.follow(next_page, callback=self.parse)

def pageRank(G, s = .85, maxerr = .0001):
    """
    Computes the pagerank for each of the n states
    Parameters
    ----------
    G: matrix representing state transitions
       Gij is a binary value representing a transition from state i to j.
    s: probability of following a transition. 1-s probability of teleporting
       to another state.
    maxerr: if the sum of pageranks between iterations is bellow this we will
            have converged.
    """
    n = G.shape[0]

    # transform G into markov matrix A
    A = csc_matrix(G,dtype=np.float)
    rsums = np.array(A.sum(1))[:,0]
    ri, ci = A.nonzero()
    A.data /= rsums[ri]

    # bool array of sink states
    sink = rsums==0

    # Compute pagerank r until we converge
    ro, r = np.zeros(n), np.ones(n)
    it = 0
    while np.sum(np.abs(r-ro)) > maxerr and it < 100:
        print(it)
        ro = r.copy()
        # calculate each pagerank at a time
        for i in range(n):
            # inlinks of state i
            Ai = np.array(A[:,i].todense())[:,0]
            # account for sink states
            Di = sink / float(n)
            # account for teleportation to state i
            Ei = np.ones(n) / float(n)

            r[i] = ro.dot( Ai*s + Di*s + Ei*(1-s) )
        it += 1

    # return normalized pagerank
    return r/float(sum(r))



def gen_matrix(list_dict):
    #print("wiki page ", len(list_dict))
    nodes = []
    seen = set()
    for ddic in list_dict:
        r_url = list(ddic.keys())[0]
        l_url = ddic.values()
        if r_url not in seen:
            seen.add(r_url)
            nodes.append(r_url)
        # for url in l_url:
        #     if url not in seen:
        #         seen.add(url)
        #         nodes.append(url)
    n = len(nodes)
    matrix = []
    # Create matix W
    # 1: visible
    # 0: invisible
    for i in range(n):
        # create list for node[i]
        val_link = []
        list_link = list(list_dict[i].values())[0]
        for j in range(n):
            tmp = list(list_dict[j].keys())[0].replace("https://en.wikipedia.org","")
            if (tmp in list_link):
                val_link.append(1)
            else:
                val_link.append(0)
        matrix.append(val_link)
    matrix = np.array(matrix)
    print(matrix[1:10])
    return matrix



In [4]:
# os.remove('result.json')
process = CrawlerProcess({
    'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)',
})

process.crawl(TestSpider)
process.start()
# Print list contain dicts 
print(type(wiki_pages))
print(pageRank(gen_matrix(wiki_pages)))

2019-04-09 22:35:45 [scrapy.utils.log] INFO: Scrapy 1.5.2 started (bot: scrapybot)
2019-04-09 22:35:45 [scrapy.utils.log] INFO: Versions: lxml 4.2.5.0, libxml2 2.9.8, cssselect 1.0.3, parsel 1.5.1, w3lib 1.20.0, Twisted 18.7.0, Python 3.7.2 (default, Dec 29 2018, 06:19:36) - [GCC 7.3.0], pyOpenSSL 18.0.0 (OpenSSL 1.1.1b  26 Feb 2019), cryptography 2.6.1, Platform Linux-4.15.0-47-generic-x86_64-with-debian-buster-sid
2019-04-09 22:35:45 [scrapy.crawler] INFO: Overridden settings: {'USER_AGENT': 'Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)'}
2019-04-09 22:35:45 [scrapy.extensions.telnet] INFO: Telnet Password: d60094a4bc9c67f7
2019-04-09 22:35:45 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.memusage.MemoryUsage',
 'scrapy.extensions.logstats.LogStats']
2019-04-09 22:35:45 [scrapy.middleware] INFO: Enabled downloader middlewares:
['scrapy.downloadermiddlewares.httpauth.HttpAuth

2019-04-09 22:35:46 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://en.wikipedia.org/wiki/Main_Page> (referer: https://en.wikipedia.org/wiki/Information_system)
2019-04-09 22:35:46 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://en.wikipedia.org/wiki/Radhika_Nagpal> (referer: https://en.wikipedia.org/wiki/Information_system)
2019-04-09 22:35:46 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://en.wikipedia.org/wiki/Data_warehouses> (referer: https://en.wikipedia.org/wiki/Information_system)
2019-04-09 22:35:46 [scrapy.core.scraper] DEBUG: Scraped from <200 https://en.wikipedia.org/wiki/Information>
{'url': 'https://en.wikipedia.org/wiki/Information', 'title': 'Information - Wikipedia'}
2019-04-09 22:35:46 [scrapy.core.scraper] DEBUG: Scraped from <200 https://en.wikipedia.org/wiki/Search_engines>
{'url': 'https://en.wikipedia.org/wiki/Search_engines', 'title': 'Web search engine - Wikipedia'}
2019-04-09 22:35:46 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://

2019-04-09 22:35:47 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://en.wikipedia.org/wiki/Business_Motivation_Model> (referer: https://en.wikipedia.org/wiki/Business_process)
2019-04-09 22:35:47 [scrapy.core.scraper] DEBUG: Scraped from <200 https://en.wikipedia.org/wiki/Departmentalization>
{'url': 'https://en.wikipedia.org/wiki/Departmentalization', 'title': 'Departmentalization - Wikipedia'}
2019-04-09 22:35:47 [scrapy.core.scraper] DEBUG: Scraped from <200 https://en.wikipedia.org/wiki/Span_of_control>
{'url': 'https://en.wikipedia.org/wiki/Span_of_control', 'title': 'Span of control - Wikipedia'}
2019-04-09 22:35:47 [scrapy.core.scraper] DEBUG: Scraped from <200 https://en.wikipedia.org/wiki/Corrective_action>
{'url': 'https://en.wikipedia.org/wiki/Corrective_action', 'title': 'Corrective and preventive action - Wikipedia'}
2019-04-09 22:35:47 [scrapy.core.scraper] DEBUG: Scraped from <200 https://en.wikipedia.org/wiki/Purchase_order>
{'url': 'https://en.wikipedia.org/wiki/

2019-04-09 22:35:48 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://en.wikipedia.org/wiki/Encyclop%C3%A9die> (referer: https://en.wikipedia.org/wiki/Business_process)
2019-04-09 22:35:48 [scrapy.core.scraper] DEBUG: Scraped from <200 https://en.wikipedia.org/wiki/Gantt_charts>
{'url': 'https://en.wikipedia.org/wiki/Gantt_charts', 'title': 'Gantt chart - Wikipedia'}
2019-04-09 22:35:48 [scrapy.core.scraper] DEBUG: Scraped from <200 https://en.wikipedia.org/wiki/Outsourcing>
{'url': 'https://en.wikipedia.org/wiki/Outsourcing', 'title': 'Outsourcing - Wikipedia'}
2019-04-09 22:35:48 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://en.wikipedia.org/wiki/Pin_(device)> (referer: https://en.wikipedia.org/wiki/Business_process)
2019-04-09 22:35:48 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://en.wikipedia.org/wiki/Workflow> (referer: https://en.wikipedia.org/wiki/Business_process)
2019-04-09 22:35:48 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://en.wikipedia.org/

2019-04-09 22:35:49 [scrapy.core.scraper] DEBUG: Scraped from <200 https://en.wikipedia.org/wiki/Task_(project_management)>
{'url': 'https://en.wikipedia.org/wiki/Task_(project_management)', 'title': 'Task (project management) - Wikipedia'}
2019-04-09 22:35:49 [scrapy.core.scraper] DEBUG: Scraped from <200 https://en.wikipedia.org/wiki/Core_business>
{'url': 'https://en.wikipedia.org/wiki/Core_business', 'title': 'Core business - Wikipedia'}
2019-04-09 22:35:49 [scrapy.core.scraper] DEBUG: Scraped from <200 https://en.wikipedia.org/wiki/Recruitment>
{'url': 'https://en.wikipedia.org/wiki/Recruitment', 'title': 'Recruitment - Wikipedia'}
2019-04-09 22:35:49 [scrapy.core.scraper] DEBUG: Scraped from <200 https://en.wikipedia.org/wiki/Management_process>
{'url': 'https://en.wikipedia.org/wiki/Management_process', 'title': 'Management process - Wikipedia'}
2019-04-09 22:35:49 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://en.wikipedia.org/wiki/Business_analysis> (referer: https://e

...

2019-04-09 22:35:59 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://en.wikipedia.org/wiki/Pacific_Asia_Conference_on_Information_Systems> (referer: https://en.wikipedia.org/wiki/Information_system)
2019-04-09 22:35:59 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://en.wikipedia.org/wiki/Americas_Conference_on_Information_Systems> (referer: https://en.wikipedia.org/wiki/Information_system)
2019-04-09 22:35:59 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://en.wikipedia.org/wiki/Information_Systems_Journal> (referer: https://en.wikipedia.org/wiki/Information_system)
2019-04-09 22:35:59 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://en.wikipedia.org/wiki/European_Journal_of_Information_Systems> (referer: https://en.wikipedia.org/wiki/Information_system)
2019-04-09 22:35:59 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://en.wikipedia.org/wiki/Journal_of_Management_Information_Systems> (referer: https://en.wikipedia.org/wiki/Information_system)
2019-04-09

2019-04-09 22:35:59 [scrapy.core.scraper] DEBUG: Scraped from <200 https://en.wikipedia.org/wiki/Software_development>
{'url': 'https://en.wikipedia.org/wiki/Software_development', 'title': 'Software development - Wikipedia'}
2019-04-09 22:35:59 [scrapy.core.scraper] DEBUG: Scraped from <200 https://en.wikipedia.org/wiki/Enterprise_Architecture>
{'url': 'https://en.wikipedia.org/wiki/Enterprise_Architecture', 'title': 'Enterprise architecture - Wikipedia'}
2019-04-09 22:35:59 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://en.wikipedia.org/wiki/Information_Science> (referer: https://en.wikipedia.org/wiki/Information_system)
2019-04-09 22:35:59 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://en.wikipedia.org/wiki/Information_Technology> (referer: https://en.wikipedia.org/wiki/Information_system)
2019-04-09 22:35:59 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://en.wikipedia.org/wiki/Cybernetics> (referer: https://en.wikipedia.org/wiki/Information_system)
2019-04-09

2019-04-09 22:36:00 [scrapy.core.scraper] DEBUG: Scraped from <200 https://en.wikipedia.org/wiki/Information_science>
{'url': 'https://en.wikipedia.org/wiki/Information_science', 'title': 'Information science - Wikipedia'}
2019-04-09 22:36:00 [scrapy.core.scraper] DEBUG: Scraped from <200 https://en.wikipedia.org/wiki/B%C3%B6rje_Langefors>
{'url': 'https://en.wikipedia.org/wiki/B%C3%B6rje_Langefors', 'title': 'Börje Langefors - Wikipedia'}
2019-04-09 22:36:00 [scrapy.core.scraper] DEBUG: Scraped from <200 https://en.wikipedia.org/wiki/Offshoring>
{'url': 'https://en.wikipedia.org/wiki/Offshoring', 'title': 'Offshoring - Wikipedia'}
2019-04-09 22:36:00 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://en.wikipedia.org/wiki/Transaction_processing_systems> (referer: https://en.wikipedia.org/wiki/Information_system)
2019-04-09 22:36:00 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://en.wikipedia.org/wiki/Chief_information_security_officer> (referer: https://en.wikipedia.org/wik

2019-04-09 22:36:01 [scrapy.core.scraper] DEBUG: Scraped from <200 https://en.wikipedia.org/wiki/Database_management_system>
{'url': 'https://en.wikipedia.org/wiki/Database_management_system', 'title': 'Database - Wikipedia'}
2019-04-09 22:36:01 [scrapy.core.engine] DEBUG: Crawled (200) <GET https://en.wikipedia.org/wiki/Information_technology> (referer: https://en.wikipedia.org/wiki/Information_system)
2019-04-09 22:36:01 [scrapy.core.scraper] DEBUG: Scraped from <200 https://en.wikipedia.org/wiki/Transaction_processing_system>
{'url': 'https://en.wikipedia.org/wiki/Transaction_processing_system', 'title': 'Transaction processing system - Wikipedia'}
2019-04-09 22:36:01 [scrapy.core.scraper] DEBUG: Scraped from <200 https://en.wikipedia.org/wiki/Organizational_informatics>
{'url': 'https://en.wikipedia.org/wiki/Organizational_informatics', 'title': 'Business informatics - Wikipedia'}
2019-04-09 22:36:01 [scrapy.core.scraper] DEBUG: Scraped from <200 https://en.wikipedia.org/wiki/Assoc

<class 'list'>
[[0 1 0 ... 0 0 0]
 [0 0 1 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
[0.00090442 0.0012413  0.00037345 0.00033995 0.00036585 0.00033685
 0.00055706 0.00035904 0.00033685 0.00039673 0.00060342 0.00033935
 0.00034411 0.00033685 0.00041789 0.0003328  0.00941509 0.0005369
 0.00194054 0.02306684 0.00674278 0.00042757 0.00040657 0.00239768
 0.00288189 0.00092446 0.00063703 0.01871464 0.00034275 0.00037231
 0.00039305 0.00034339 0.00033451 0.00033286 0.00035333 0.00039008
 0.00034208 0.00054053 0.0003346  0.00033456 0.00036604 0.00034992
 0.00036541 0.00044197 0.00043285 0.00037464 0.00038964 0.00035986
 0.00039968 0.00320198 0.00480782 0.0013

In [5]:
a = ['/wiki/Information_system']

In [6]:
'/wiki/Information_system' in a

True